In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd

In [2]:
dataset = tf.keras.utils.image_dataset_from_directory(
    'data',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True
)
dataset = dataset.map(lambda x, y: (x / 255.0, y))

Found 536 files belonging to 6 classes.


In [5]:
X_data, y_data = [], []
for batch in dataset:
    X, y = batch
    X_data.append(X.numpy())
    y_data.append(y.numpy())
X_data = np.concatenate(X_data)
y_data = np.concatenate(y_data)
print("Image shape before flattening:", X_data.shape)

Image shape before flattening: (536, 224, 224, 3)


In [7]:
X_flat = X_data.reshape(len(X_data), -1)
print("Shape after flattening:", X_flat.shape)

Shape after flattening: (536, 150528)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_flat, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [11]:
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC()
}
results = []

In [13]:
for name, clf in models.items():
    print(f"Training {name}...")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} => Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}\n")
    results.append((name, precision, recall, accuracy))


Training RandomForest...
RandomForest => Precision: 0.7318, Recall: 0.7315, Accuracy: 0.7315

Training SVM...
SVM => Precision: 0.7809, Recall: 0.7593, Accuracy: 0.7593


In [17]:
df = pd.DataFrame(results, columns=["Model", "Precision", "Recall", "Accuracy"])
print(df)
df.to_csv("classical_model_results.csv", index=False)

          Model  Precision    Recall  Accuracy
0  RandomForest   0.731790  0.731481  0.731481
1           SVM   0.780945  0.759259  0.759259
